In [48]:
import pathlib as pl
import pandas as pd
import re
import collections as col

pd.set_option('future.no_silent_downcasting', True)

%cd -q "/home/ebertp/work/code/cubi/project-run-hgsvc-hybrid-assemblies/notebooks"
_PROJECT_CONFIG_NB = str(pl.Path("00_project_config.ipynb").resolve(strict=True))

%run $_PROJECT_CONFIG_NB

_MYNAME="clean-other-data-tables"
_NBSTAMP=get_nb_stamp(_MYNAME)

all_data_tables = [
    ("HiC", [
        PROJECT_BASE.joinpath("annotations", "external", "accessions", "20240521_hgsvc_HiC_runs.PH.tsv"),
        PROJECT_BASE.joinpath("annotations", "external", "accessions", "20240805_HiC-MissingFiles_filereport_read_run_PRJEB39684.FY.csv"),
        #PROJECT_BASE.joinpath("annotations", "external", "accessions", "20240805_HGSVC3-HiC_missingSamples.FY.csv"),
        PROJECT_BASE.joinpath(
            "annotations", "external", "accessions",
            "20240809_HGSVC3-HiC_missingSamples_rawreadsubmission_shared_v2.FY.csv"
        ),
        PROJECT_BASE.joinpath(
            "annotations", "external", "accessions",
            "PRJEB39684_filereport_HiC_ENA.tsv"
        ),
        PROJECT_BASE.joinpath(
            "annotations", "external", "accessions",
           "PRJEB58376_ENA_filereport.HiC.tsv"
        ),
    ]
    ),
    ("IsoSeq", [PROJECT_BASE.joinpath("annotations", "external", "accessions", "20240521_hgsvc_isoseq_runs.PH.tsv")]),
    ("RNASeq", [
        PROJECT_BASE.joinpath("annotations", "external", "accessions", "20240521_hgsvc_rnaseq_runs.PH.tsv"),
        #PROJECT_BASE.joinpath("annotations", "external", "accessions", "20240805_HGSVC3_RNASeq_missingSamples.FY.csv"),
        PROJECT_BASE.joinpath(
            "annotations", "external", "accessions",
            "20240813_HGSVC3_RNASeq_missingSamples_rawreadsubmission_shared_v2.FY.csv"
        )
    ]
    ),
]

keep_columns = [
    "accession", "project_accession", "sample_alias", "library_name", "file_name"
]

def assert_sample_id(sample, include_sex=True):

    if sample in HGSVC_SAMPLES["sample"].values:
        norm_sample = sample
    else:
        if sample in ["GM19320", "NA19320", "HG02016"]:
            norm_sample = None
        elif sample in ["NA19219", "GM19219"]:
            norm_sample = "NA19129"
        elif sample == "HG02818-replacement":
            norm_sample = "HG02818"
        elif sample.replace("GM", "NA") in HGSVC_SAMPLES["sample"].values:
            norm_sample = sample.replace("GM", "NA")
        else:
            raise ValueError(sample)

    if norm_sample is not None:
        sex = HGSVC_SAMPLES.loc[HGSVC_SAMPLES["sample"] == norm_sample, "sex"].iloc[0]
    else:
        sex = None

    if include_sex:
        return norm_sample, sex
    else:
        return norm_sample


def normalize_columns(table_columns):

    keepers = []
    renamer = dict()

    for column in table_columns:
        if column == "id":
            renamer[column] = "accession"
            keepers.append(column)
        if column == "project_accession":
            keepers.append(column)
        if column == "library_name":
            renamer[column] = "sample"
            keepers.append(column)
        if column == "file_name":
            keepers.append(column)
        if column == "submitted_ftp":
            keepers.append(column)
            renamer[column] = "file_name"
        if column == "run_accession":
            keepers.append(column)
            renamer[column] = "accession"
        if column == "study_accession":
            keepers.append(column)
            renamer[column] = "project_accession"

    return keepers, renamer


isoseq_samples = """
NA18989
HG03248
HG01457
NA19331
HG02666
HG02106
HG04217
NA19317
NA19347
HG03807
HG00268
NA19384
"""

hic_isoseq_samples = sorted(isoseq_samples.strip().split())

for data_label, data_tables in all_data_tables:
    #if data_label != "HiC":
    #    continue
    print("=============")
    print(data_label)
    concat = []
    for data_table in data_tables:
        print(data_table.name)
        if data_table.name.endswith(".tsv"):
            sep = "\t"
        else:
            sep = ","
        table = pd.read_csv(data_table, sep=sep, header=0)
        keep_columns, rename_columns = normalize_columns(table.columns)
        table = table[keep_columns].copy()
        table.rename(rename_columns, axis=1, inplace=True)
        table["metadata_source"] = data_table.name

        try:
            table["file_name"] = table["file_name"].apply(lambda fn: pl.Path(fn).name)
        except TypeError:
            print(table)
            print(table.columns)
            raise

        try:
            sample_sex = table["sample"].apply(assert_sample_id)
        except KeyError:
            print("Extract sample ID from filename")
            table["sample"] = table["file_name"].apply(lambda fn: pl.Path(fn).name.split("_")[0].replace("GM", "NA"))
            sample_sex = table["sample"].apply(assert_sample_id)
        selector = [t[0] is not None for t in sample_sex]
        table = table.loc[selector, :]
        table.reset_index(drop=True, inplace=True)
        table["sample"] = table["sample"].apply(assert_sample_id, args=(False,))
        sample_sex = [t for t in sample_sex if t[0] is not None]
        sample_sex = pd.DataFrame.from_records(
            sample_sex, index=table.index, columns=["sample", "sex"]
        )
    
        table = table.merge(sample_sex, on="sample", how="outer")
        table.set_index(["sample", "sex"], inplace=True)
        table.insert(0, "data_type", data_label)
        table.sort_index(inplace=True)
        concat.append(table)

    if len(concat) > 1:
        table = pd.concat(concat, axis=0, ignore_index=False)
    else:
        table = concat[0]

    if data_label in ["IsoSeq"]:
        table = table[table.index.get_level_values("sample").isin(hic_isoseq_samples)]
    
    table.sort_index(inplace=True)
    table.drop("metadata_source", axis=1, inplace=True)
    table.drop_duplicates(["project_accession", "file_name"], inplace=True, keep="first")
    
    if pd.isnull(table["accession"]).any():
        missing = table.loc[pd.isnull(table["accession"]), :]
        for row in missing.itertuples():
            print(row.Index, row.data_type, row.project_accession)
            print(row.file_name)
            print("===")
        raise ValueError("missing file accession")
    if pd.isnull(table["project_accession"]).any():
        missing = table.loc[pd.isnull(table["project_accession"]), :]
        for row in missing.itertuples():
            print(row.Index, row.data_type, row.project_accession)
            print(row.file_name)
            print("===")
        raise ValueError("missing project accession")
    out_file_name = f"table_SXPE_{data_label}_accessions.tsv"
    out_file_path = TABLE_OUT_SUPPL.joinpath(out_file_name)
    table.to_csv(out_file_path, sep="\t", header=True, index=True)
    sample_count = col.Counter(table.index.get_level_values("sample"))
    total_samples = set(table.index.get_level_values("sample"))
    print("samples in table ", len(total_samples))       

    for sample in HGSVC_SAMPLES["sample"].values:
        if sample not in total_samples:
            print("missing ", sample)
    df = pd.read_csv(out_file_path, sep="\t", header=0)
    df.drop_duplicates("sample", inplace=True)
    print(df.shape[0])
    print(df["project_accession"].value_counts())
    

HiC
20240521_hgsvc_HiC_runs.PH.tsv
20240805_HiC-MissingFiles_filereport_read_run_PRJEB39684.FY.csv
Extract sample ID from filename
20240809_HGSVC3-HiC_missingSamples_rawreadsubmission_shared_v2.FY.csv
PRJEB39684_filereport_HiC_ENA.tsv
PRJEB58376_ENA_filereport.HiC.tsv
samples in table  60
missing  NA19238
missing  HG00732
missing  HG00513
missing  NA24385
missing  HG02818
60
project_accession
PRJEB39684    30
PRJEB75193    20
PRJEB58376    10
Name: count, dtype: int64
IsoSeq
20240521_hgsvc_isoseq_runs.PH.tsv
samples in table  12
missing  NA19238
missing  NA19239
missing  NA19240
missing  HG00731
missing  HG00732
missing  HG00733
missing  HG00512
missing  HG00513
missing  HG00514
missing  HG03732
missing  HG01573
missing  HG00171
missing  HG02587
missing  HG01114
missing  HG02018
missing  NA19036
missing  NA12329
missing  HG02011
missing  NA19983
missing  HG03371
missing  HG02492
missing  HG03009
missing  HG03683
missing  HG03065
missing  NA18534
missing  HG01505
missing  NA19650
missin